In [7]:
import torch
import matplotlib.pyplot as plt 
import numpy as np

words=open('names.txt','r').read().splitlines()

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s,i in stoi.items()}

xs,ys=[],[]
for w in words:
    chs=['.']+list(w)+['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs=torch.tensor(xs)
ys=torch.tensor(ys)
num=xs.nelement()
print('num samples: ',num)

W=torch.randn((27,27),requires_grad=True)
import torch.nn.functional as F
xenc = F.one_hot(xs,num_classes=27).float()

for k in range(100):
    xenc=F.one_hot(xs,num_classes=27).float()
    logits=xenc@W
    counts=logits.exp()
    probs = counts / counts.sum(1,keepdims=True)
    loss = -probs[torch.arange(num),ys].log().mean()
    print(loss.item())

    W.grad=None
    loss.backward()

    W.data+= -10*W.grad

num samples:  228146
3.748014450073242
3.665358781814575
3.590296983718872
3.5219736099243164
3.4596996307373047
3.4028892517089844
3.3510243892669678
3.303631067276001
3.2602620124816895
3.220493793487549
3.1839280128479004
3.150205373764038
3.1190075874328613
3.0900626182556152
3.0631420612335205
3.0380523204803467
3.0146312713623047
2.9927382469177246
2.972252130508423
2.9530649185180664
2.935080051422119
2.918208122253418
2.9023685455322266
2.887486457824707
2.873491048812866
2.8603177070617676
2.8479068279266357
2.8362016677856445
2.8251497745513916
2.814703941345215
2.804819107055664
2.795454263687134
2.786571502685547
2.7781364917755127
2.7701175212860107
2.7624847888946533
2.7552120685577393
2.7482738494873047
2.7416481971740723
2.7353146076202393
2.7292535305023193
2.7234480381011963
2.7178823947906494
2.712541341781616
2.707411050796509
2.7024800777435303
2.6977362632751465
2.693169116973877
2.6887688636779785
2.684526205062866
2.6804323196411133
2.676480293273926
2.672661542

In [10]:
def generate_name(W, stoi, itos, max_len=15):
    name = ['.']
    while len(name) < max_len:
        prev_char = name[-1]
        prev_ix = torch.tensor([stoi[prev_char]])
        prev_one_hot = F.one_hot(prev_ix, num_classes=27).float()
        logits = prev_one_hot @ W
        probs = F.softmax(logits, dim=1).detach().numpy().squeeze()
        next_char = np.random.choice(list(itos.values()), p=probs)
        if next_char == '.':
            break
        name.append(next_char)
    return ''.join(name[1:])

generated_names = [generate_name(W, stoi, itos) for _ in range(10)]
print(generated_names)

['', 'laembesyostphv', 'njljbkfr', 'cjbfsayqaohbjb', 'cmbwqvfojbxeas', 'qfmcrpsjbsyeya', 'kawvbsdbsbgfxa', 'uxejptfvjbttbb', 'knagfxnzdhaooo', 'tzbjpzbxg']
